1. why p-1?
2. 是否需要存储公钥私钥然后再进行环签名
3. 改成DSA lib中的公私钥加密和随机数生成

In [154]:
import random
from hashlib import sha256

def generate_keys(g, p):
    # 生成私钥s
    s = random.randint(1, p-2)
    # 计算公钥v = g^s mod p
    v = pow(g, s, p)
    return s, v

def schnorr_sign(message, g, p, s):
    # 选择随机数k
    k = random.randint(1, p-2)
    # 计算r = g^k mod p
    r = pow(g, k, p)
    # 将消息和r的值哈希，然后与私钥s结合产生签名
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    x = (k - s * hash_value) % (p-1)
    return r, x

def schnorr_verify(message, g, p, v, r, x):
    # 重新计算r的哈希值
    hash_value = int(sha256((str(r) + message).encode()).hexdigest(), 16)
    # 使用公钥v和签名x、r计算验证值
    rv = (pow(g, x, p) * pow(v, hash_value, p)) % p
    return rv == r

# 设置大素数p和其原根g
p = 1000000007  # 一个大素数
g = 5  # p的原根

# 生成密钥
s, v = generate_keys(g, p)

# 签名消息
message = "Hello, Schnorr!"
r, x = schnorr_sign(message, g, p, s)
print(f"Signature: (r={r}, x={x})")

# 验证签名
valid = schnorr_verify(message, g, p, v, r, x)
print(f"Signature valid: {valid}")


Signature: (r=756185547, x=149736893)
Signature valid: True


In [155]:
import random 
from hashlib import sha256


p = 1000000007  # prime
g = 5  # primitive root of p

def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(pow(g, r, p))).encode()).hexdigest(), 16)
print('e3:', e3)

_, random_s3 = generate_keys(g, p)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e1:', e1)

_, random_s1 = generate_keys(g, p)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
print('e2:', e2)

# why p-1?
s2 = (r - x2 * e2) % (p-1)
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3
}

print('signature process total result:')
display(total_res)

msg: Hello, Ring Signature
pubkeys: [417972493, 968634488, 805984085]
Y: 417972493968634488805984085
e3: 23299363187223293576147029669753009102510702700275497468033058576890608831573
random_s3: 470576925
e1: 60246566526499533121676458949285734704336613253755169601860474893228677194701
random_s1: 175159952
e2: 68270066951973315249926724473378211566265554608335630336960469803975069934735
s2: 821950262
signature process total result:


{'e1': 60246566526499533121676458949285734704336613253755169601860474893228677194701,
 's1': 175159952,
 's2': 821950262,
 's3': 470576925}

In [156]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(calculation)).encode()).hexdigest(), 16)

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)
print(e2_V==e2)
print(e3_V==e3)

60246566526499533121676458949285734704336613253755169601860474893228677194701
68270066951973315249926724473378211566265554608335630336960469803975069934735
23299363187223293576147029669753009102510702700275497468033058576890608831573
True
True
True


In [157]:
e3_calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_calculation

494496705

In [158]:
e3_original_calculation = pow(g, r, p) % p
e3_original_calculation

494496705

In [159]:
(pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p

494496705

In [160]:
pow(g, r, p)

494496705

In [161]:
import random
from hashlib import sha256

p = 1000000007  # prime
g = 5  # primitive root of p

def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v

def generate_tag(x, g, p):
    # Generate tag using private key x and generator g
    tag = pow(g, x, p)
    return tag

msg = 'Hello, Ring Signature'
print('msg:', msg)

x1, y1 = generate_keys(g, p)
x2, y2 = generate_keys(g, p)
x3, y3 = generate_keys(g, p)
public_key_list = [y1, y2, y3]
print('pubkeys:', public_key_list)
Y = ''.join([str(i) for i in public_key_list])
print('Y:', Y)

tag = generate_tag(x1, g, p)  # Use x1 for the tag, can be any x
print('tag:', tag)

r, R = generate_keys(g, p)
e3 = int(sha256((str(msg) + str(Y) + str(tag) + str(pow(g, r, p))).encode()).hexdigest(), 16)
print('e3:', e3)

_, random_s3 = generate_keys(g, p)
print('random_s3:', random_s3)

calculation = (pow(g, random_s3, p)*pow(y3, e3, p)) % p
e1 = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
print('e1:', e1)

_, random_s1 = generate_keys(g, p)
print('random_s1:', random_s1)

calculation = (pow(g, random_s1, p)*pow(y1, e1, p)) % p
e2 = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
print('e2:', e2)

# why p-1?
s2 = (r - x2 * e2) % (p-1)
print('s2:', s2)

total_res = {
    'e1': e1,
    's1': random_s1,
    's2': s2,
    's3': random_s3,
    'tag': tag
}

print('signature process total result:')
display(total_res)


msg: Hello, Ring Signature
pubkeys: [375738166, 448155835, 597705860]
Y: 375738166448155835597705860
tag: 375738166
e3: 814834924727847369884819828522728757321599742227078323596485851524865993759
random_s3: 287892895
e1: 14504380731847738628765761827553660089486291504760028699187853653962702233568
random_s1: 613868940
e2: 50964452774902056429131073473492124925558970319187421142422880537378964686574
s2: 555720001
signature process total result:


{'e1': 14504380731847738628765761827553660089486291504760028699187853653962702233568,
 's1': 613868940,
 's2': 555720001,
 's3': 287892895,
 'tag': 375738166}

In [162]:
calculation = (pow(g, total_res['s1'], p)*pow(y1, total_res['e1'], p)) % p
e2_V = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s2'], p) * pow(y2, e2_V, p)) % p
e3_V = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)
calculation = (pow(g, total_res['s3'], p)*pow(y3, e3_V, p)) % p
e1_V = int(sha256((str(msg) + str(Y) + str(tag) + str(calculation)).encode()).hexdigest(), 16)

print(e1_V)
print(e2_V)
print(e3_V)
print(e1_V==e1)
print(e2_V==e2)
print(e3_V==e3)

14504380731847738628765761827553660089486291504760028699187853653962702233568
50964452774902056429131073473492124925558970319187421142422880537378964686574
814834924727847369884819828522728757321599742227078323596485851524865993759
True
True
True
